# VeraGPT (Mistral-Core) Test
Run this notebook in Google Colab to load the Mistral-7B-Instruct model and test prompts.

In [ ]:
# Install dependencies (Colab)
!pip -q install "transformers>=4.45.0" "accelerate>=0.34.0" "torch>=2.1.0" "python-dotenv>=1.0.1"

In [ ]:
# Optional: set your Hugging Face token (needed for gated models)
import os
os.environ["HF_TOKEN"] = ""  # <- paste your token or leave blank if not required
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

token = os.environ.get("HF_TOKEN") or None
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=token, use_fast=True)

device_map = "auto" if torch.cuda.is_available() else None
dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    token=token,
    torch_dtype=dtype,
    device_map=device_map,
)
if device_map is None:
    model.to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

In [ ]:
def generate_response(user_prompt: str, system_prompt: str = "You are a helpful assistant.") -> str:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    if hasattr(tokenizer, "apply_chat_template"):
        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    else:
        prompt = f"[SYSTEM] {system_prompt}\n[USER] {user_prompt}\n[ASSISTANT]"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output_ids = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.95,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
    )
    generated = output_ids[0][inputs["input_ids"].shape[1] :]
    return tokenizer.decode(generated, skip_special_tokens=True).strip()

In [ ]:
prompt = "Explain zero-shot reasoning in simple terms."
print(generate_response(prompt))